In [4]:
import h5py
from sklearn.metrics import mean_squared_error as mse
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import lightgbm as lgb
import warnings

from DNN import *
from processing_results import * 
from training import *
from stacking_models import *

from data_preprocessing.samples import *
# Warning because some galaxies (those are not of orur interest have mass/ halo concentration 0, which we logarithmize)
warnings.filterwarnings("ignore")

%matplotlib inline
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'data_preprocessing'

In [ ]:
subhalos, halos, halo_concentration = Load_TNG() # IllustrisTNG data should be added to the 'data' folder first
ELGs = main(5*861, "ELG", subhalos, halos, halo_concentration, sSFR=9.086, max_conc=100)
LRGs = main(2*861, "LRG", subhalos, halos, halo_concentration, sSFR=9.086, max_conc=100)

In [ ]:
ELGs["g-r"] = ELGs["lum_g"]-ELGs["lum_r"]
LRGs["g-r"] = LRGs["lum_g"]-LRGs["lum_r"]

features = ["crit200", "crit500", "mean200", "grouop_vel"]
ELGs[["crit200", "crit500", "mean200"]] = np.log(ELGs[["crit200", "crit500", "mean200"]])
LRGs[["crit200", "crit500", "mean200"]] = np.log(LRGs[["crit200", "crit500", "mean200"]])

In [ ]:
train, test = train_test(ELGs, val=False)
act = test["g-r"].values

# Use simple model without optimizing parameters just to see approximately how good the predictions will be
model = lgb.LGBMRegressor()
model.fit(train[features],train["g-r"])
pred = model.predict(test[features])

In [ ]:
plt.title("ELG, n=7e-4, g-r predictions from 3 halo masses and halo group veloity") #,fontweight="bold"
plt.scatter(pred, act, label=f"RMSE={round(np.sqrt(mse(pred,act_elg)),2)}",s=4)
plt.legend()
plt.grid(lw=0.2)
plt.xlabel("Predicted")
plt.ylabel("Actual")
x = np.linspace(-0.1,0.3,10)
plt.plot(x,x,lw=2,c="r")